# Boyd and Vandenberghe - Convex Optimisation
## Chapter 9 - Unconstrained Optimisation

### Golden Section Search

In this notebook I discuss the implementation of golden section search as covered [here](https://jpivan.github.io/boyd-convex-optimisation/ch9/).

As of this moment this function has only been tested a little, I will be covering writing tests later. If any problems are revealed in testing they will be noted here and this notebook updated.

In [1]:
%%capture
# just some setup, don't worry too much
from os import getcwd
while getcwd().split('\\')[-1] != 'optimisation':
    %cd ..

%load_ext autoreload
%autoreload 2

# we need numpy for the functions we will copy from the optimisation repo

import numpy as np

So, let's load the function from the source file and take a look at it. (It's loaded in the cell below.)

The first thing to mention is a 'design' choice I made. The function for golden section seach and backtracking search should expose a similar interface. That is, a function designed to call one of them should also work when calling the other. For now I have decided that the functions should return a dictionary, containing `'t'`, the step size, and `'x'`, the value of $x + t\Delta x$. Offloading this responsibility to the line search functions means that a function which uses them does not need to care whether it is calling a line search method that calculates an optimum $x^*$ (like golden section search) or a recommended step size $t$ (like backtracking) since all line search functions will by design return both.

We then define the constants we will be using in each iteration of the search: $\varphi^{-1}$ and $\varphi^{-2}$. We also initialise a counter `nfev` which will track how many function evaluations the line search performs. This is a very important statistic in many cases. In order to actually keep track of the function evaluations we wrap `func` inside another function `_f`, which increments this counter before returning `func(x)`. I don't know if this is particularly "pythonic", but it seems like a good way to do it. It creates an extra stack frame, but if that kind of detail is going to make or break your solution I doubt python is the right tool for the job.

We are now at `_gs`, which is where the actual line search will occur. Why should we create this "hidden" function instead of just making `goldensection` the recursive line search function?

Well, if a function wishes to utilise backtracking line search it should provide a starting point and a search direction. Golden section search is not derived in the same way! Golden section search requires an interval containing a minimum. If we wish the two functions to be interchangable we must address this disparity. Furthermore `_gs` must be able to take a whole list of arguments, $x_2$, $x_3$, $fx_2$, $fx_3$ which a caller is unlikely to be able to provide anyway. So, we wrap up `_gs` and hide the details from the caller so that it is more easliy interchangeable with a backtracking line search function.

If you follow the pseudocode we wrote earlier you will see that `_gs` is defined exactly as we discussed with one small exception:

```if np.linalg.norm(h) <= precision: return (x1 + x4) / 2```

Why take the norm of `h` instead of just using `h` directly? Although line search occurs in one dimension, we are usually interested in $x \in \mathbb{R}^n$, so we need to be a little cognisant of any points in our implementation where the fact that $x$ may be a numpy array may be relevant.

The last few lines bracket the minimum (very lazily). This is enclosed in a `for`-`else` loop that will raise an error if we fail to bracket the minimum within a reasonable time. We then call `_gs` to find the minimum. When we return our result the line `'t': np.average((xopt - x) / dx)` is a bit of a hack. $(x^*-x) / \Delta x$ should in principle be exactly $t\mathbf{1}$, but I decided to take an average just in case of some precision loss on one of the entries. This may prove to be a bad idea. We will find out in testing!

In [2]:
# %load -s goldensection src/line_search.py
def goldensection(func, x, dx, precision=1e-6):
    """ Golden-section search. Assumes minimum is in the direction dx,
    starting at x.

    Args:
        func: function being minimised
        x: start point
        dx: search direction
        precision: uncertainty permitted in result

    Returns:
        't': argmin_s f(x + s*dx)
        'x': optimal x found by golden section search
        some other metadata
    """
    iphi = (5**0.5 - 1 )/ 2  # 1/phi
    iphi2 = (3 - 5**0.5) / 2  # 1/phi^2
    nfev = 0

    def _f(x):
        nonlocal nfev
        nfev += 1
        return func(x)

    def _gs(x1, x4, h=None, x2=None, x3=None, fx2=None, fx3=None):
        # We are going to divide the search space into three sections with
        # boundaries (x1, x2, x3, x4) and perform golden section search.
        # Function values are saved between iterations. 
        if h is None: h = x4 - x1
        if np.linalg.norm(h) <= precision: return (x1 + x4) / 2
        if x2 is None: x2 = x1 + iphi2*h
        if x3 is None: x3 = x1 + iphi*h
        if fx2 is None: fx2 = _f(x2)
        if fx3 is None: fx3 = _f(x3)

        if fx2 < fx3:
            return _gs(x1, x3, h=h*iphi, x3=x2, fx3=fx2)
        else:
            return _gs(x2, x4, h=h*iphi, x2=x3, fx2=fx3)

    # first we need to bracket the minimum
    t = 1
    for _ in range(64):
        if _f(x + t*dx) > _f(x):
            break
        t *= 2
    else:
        raise RuntimeError(
            "Bracketing minimum failed. "
            "Check if func is convex, start point, and search direction."
        )
    # minimum is now definitely between f(x + t*dx) and f(x)
    # do golden section search
    xopt = _gs(x, x + t*dx)  # golden section actually finds the optimal x
    
    return {
        'x': xopt,
        't': np.average((xopt - x) / dx),  # x* = x + t*dx
        'nfev': nfev,
    }


In creating this function we have used several tools that those new to python or programming in general might not be familiar, or comfortable, with.

In order of appearance we have:
- the `nonlocal` keyword
- a `for`-`else` loop
- a call to a recursive function

### The `nonlocal` Keyword

Beginning with the `nonlocal` keyword, in the following block the identifier `var` is in the global scope in this notebook. All functions at all levels can 'see' (access) the object (in this case a `list`) that `var` refers to (after it has been declared, the functions preceding it cannot access it before it exists!).

In [3]:
var = [1, 2, 3]

def print_var():
    print(var)

def modify_var():
    var[2] += var[2]

This is important, especially for things like lists which are *mutable* (compare *immutable*), that is, which can be changed.

In [4]:
print(var)
print_var()

modify_var()

print(var)
print_var()

Notice that the global `var` is altered when `modify_var` is called.

In [5]:
def i_make_my_own_var():
    var = "This is not your global var, but my local var."
    print(var)

i_make_my_own_var()
print(var)
modify_var()
print_var()

Notice that `i_make_my_own_var`, well, makes its own local copy of `var` that is distinct from the global `var`, but it does not *globally* overwrite it. However, any functions defined in this scope can no longer directly acess the global `var`. See below, where `i_access_local_var` does not print the list in the global `var`, likewise we can modify the local copy of `var` as you might expect.

In [6]:
def outer_function():
    var = ["local", "var"]
    print(var)

    def i_access_local_var():
        print(var)

    def i_modify_local_var():
        var[0] = "modified local"
        print(var)

    i_access_local_var()
    i_modify_local_var()

outer_function()
print(var)

So why does this not work?

In [7]:
def outer_function():
    var = ["local", "var"]
    print(var)

    def i_fail():
        var = var*2
        print(var)

    i_fail()

outer_function()


['local', 'var']


UnboundLocalError: local variable 'var' referenced before assignment

In the function `i_fail` we create a local identifier `var` which *blocks us from referring to the `var` in the enclosing scope using the same identifier*. So in `var = var*2` we are referring to `var` from within `i_fail` on the right-hand side of the assignment (=) as well. Since the local identifier `var` in `i_fail` is not bound to anything yet we cannot evaluate `var*2` and the error results.

In [8]:
def outer_function():
    var = ["local", "var"]
    print(var)  # local var in outer_function

    def i_extend_the_list_in_outer_function():
        nonlocal var
        var = var + ['modified']
        print(var)

    def i_extend_the_list_in_global_scope():
        global var
        var = var + ['modified the global!']

    i_extend_the_list_in_outer_function()  # modifies and prints local var in outer_function
    i_extend_the_list_in_global_scope()  # modifies global var
    print(var)  # prints local var again to show that i_extend_the_list_in_global_scope has left it untouched

outer_function()
print(var)

['local', 'var']
['local', 'var', 'modified']
['local', 'var', 'modified']
[1, 2, 12, 'modified the global!']


The `nonlocal` and `global` keywords can be used to cause listed identifiers (`nonlocal var1, var2` is valid) to refer to previously bound identifiers in the enclosing scope, or in the global scope. To my knowledge there is no non-hacky way to refer to variables in an intermediate scope. This is probably a good thing, since having a variable with the same identifier in multiple scopes can already lead to confusion and bugs if abused. If you find yourself needing a structure like the one below, and require `c` to refer to `var` as defined in `a` you probably want to revise your method rather than looking for a hacky way to get around this "limitation".

In [9]:
def a():
    var = ["a's var"]
    def b():
        var = ["b's var"]
        def c():
            nonlocal var
            var += ["c's addition"]
            print(var)
        c()
    b()
a()

["b's var", "c's addition"]


### `for`-`else` Loops

`for`-`else` loops probably have less potential for confusion than scoping rules. A `for`-`else` loop executes the `else` block if the loop complete without breaking. In the block below the first loop terminates without encountering a `break` statement. (`_` is a common name used for throwaway variables in Python.) The second loop terminates when `i == 5` triggers the `break`, and so the `else` block does not execute.

In [10]:
for _ in range(10):
    pass # do nothing
else:
    print("Loop 1 terminated nomally.")

for i in range(10):
    if i == 5: break
else:
    print("Loop 2 terminated nomally.")

Loop 1 terminated nomally.


### Recursion

I remember the discomfort of learning to use recursion and the endless errors, stack overflows, and memory leaks that came with it. I have friends and colleagues that have been programming for a few years and still would rather die on the hill of multiply-nested loops several times before even considering a recursive algorithm. To be clear they are not developers or software engineers, so this is by no means unforgivable.

By all means, it is possible to write a recursive algorithm for problems that are not well-suited to such a formulation and I suspect that some of the bad experiences of learning to use recursion might come from such examples where recursion created unnecessary complexity. That said, some structures and problems are "inherently" recursive and recognising them as such can often simplify them, rather than complicate them.

If you find the lines `return _gs(x1, x3, h=h*iphi, x3=x2, fx3=fx2)` and `return _gs(x2, x4, h=h*iphi, x2=x3, fx2=fx3)` intimidating because the function `_gs` calls itself in its return statement then perhaps a little time spent learning about recursion will help.

I may create some notes on the topic at a later date and they will be linked here once they exist. In the meantime I think this [medium article](https://medium.com/analytics-vidhya/brief-introduction-to-recursion-8ea409b5f1bf) is a good introductory read.